# Install All the Required Packages

In [1]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets

In [14]:
# pip install tensorboardX

# Import All the Required Libraries for fine-tuning Llama2

We would be performing [Q-LoRA fine-tuning](https://huggingface.co/blog/4bit-transformers-bitsandbytes). QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

1. Load a llama-2-7b-chat-hf model (chat model)
2. Train it on the Arnab13/guanaco-llama2-reverse-instruct, which will produce our fine-tuned model Llama2-7b-BackInstruct

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "Arnab13/guanaco-llama2-reverse-instruct"

# Fine-tuned model name
new_model = "Llama2-7b-BackInstruct"

In [4]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [5]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [6]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

In [7]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Start the fine-tuning process

1. We load the instruction backtranslation dataset 
2. Then, we’re configuring bitsandbytes for 4-bit quantization.
3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.
4. Finally, we're loading configurations for QLoRA, regular training parameters, and passing everything to the SFTTrainer. The training can finally start!

In [8]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.42s/it]
/home/jovyan/.local/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/jovyan/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [9]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.496100
50,1.518000
75,1.375700
100,1.294400
125,1.475800
150,1.177000


TrainOutput(global_step=162, training_loss=1.4111063333205234, metrics={'train_runtime': 116.1378, 'train_samples_per_second': 5.571, 'train_steps_per_second': 1.395, 'total_flos': 4061995356487680.0, 'train_loss': 1.4111063333205234, 'epoch': 1.0})

In [10]:
# Save trained model
trainer.model.save_pretrained(new_model)

# Let's test out the Instruction Backtranslation model

We will use the text generation pipeline to provide output and check the instruction generated

In [63]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [64]:
# Run text generation pipeline with our next model
prompt = '''A penguin and a walrus walk into a bar. The bartender looks up and says, "What is this, some kind of Arctic joke?" The penguin and walrus just look at each other, confused. Then the walrus shrugs and says, "I don't know about Arctic jokes, but we sure know how to break the ice!" The penguin rolls his eyes, but can't help but chuckle.'''

In [65]:
result = pipe(f"<s>[INST] {prompt} [/INST]")

In [66]:
print(prompt)

A penguin and a walrus walk into a bar. The bartender looks up and says, "What is this, some kind of Arctic joke?" The penguin and walrus just look at each other, confused. Then the walrus shrugs and says, "I don't know about Arctic jokes, but we sure know how to break the ice!" The penguin rolls his eyes, but can't help but chuckle.


In [67]:
print(result[0]['generated_text'].split('[/INST]')[-1])

 I want to make a joke about penguins and walruses walking into a bar, but I can't think of anything. Can you help me come up with a joke? 


In [68]:
prompt = '''There are many great canvas libraries for browser JavaScript. Some popular choices include: 1. D3.js: a powerful library for data visualization and creating interactive graphics. 2. Particles.js: a lightweight library for creating particle effects and animations. 3. Three.js: a 3D library for creating interactive and immersive experiences. 4. Konva.js: a high-performance library for creating complex canvas graphics and animations. 5. Paper.js: a vector graphics library for creating complex geometric shapes and animations. 6. Fabric.js: a powerful library for working with SVG and canvas graphics, including support for user interaction. 7. Pixi.js: a fast and lightweight 2D rendering engine for creating interactive graphics and games. 8. Chart.js: a popular library for creating charts and graphs from data. Each library has its own strengths and weaknesses, so the choice of which to use will depend on the specific needs of your project. It's important to research and evaluate each library carefully to ensure that it meets your requirements and fits well with your development workflow.'''

In [69]:
result = pipe(f"<s>[INST] {prompt} [/INST]")

In [70]:
print(prompt)

There are many great canvas libraries for browser JavaScript. Some popular choices include: 1. D3.js: a powerful library for data visualization and creating interactive graphics. 2. Particles.js: a lightweight library for creating particle effects and animations. 3. Three.js: a 3D library for creating interactive and immersive experiences. 4. Konva.js: a high-performance library for creating complex canvas graphics and animations. 5. Paper.js: a vector graphics library for creating complex geometric shapes and animations. 6. Fabric.js: a powerful library for working with SVG and canvas graphics, including support for user interaction. 7. Pixi.js: a fast and lightweight 2D rendering engine for creating interactive graphics and games. 8. Chart.js: a popular library for creating charts and graphs from data. Each library has its own strengths and weaknesses, so the choice of which to use will depend on the specific needs of your project. It's important to research and evaluate each library

In [71]:
print(result[0]['generated_text'].split('[/INST]')[-1])

 What are some great canvas libraries for browser JavaScript? 


##### Observation: Looks like our Backtranslation model does generated relevant instructions

## Push the model to HuggingFace Hub

In [61]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

We need to merge the weights from LoRA with the base model.

In [62]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.11s/it]


Our weights are merged and we reloaded the tokenizer. We can now push everything to the Hugging Face Hub to save our model.

In [74]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model.push_to_hub("Llama2-7b-BackInstruct", check_pr=True)
tokenizer.push_to_hub("Llama2-7b-BackInstruct",check_pr=True)